In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from IPython.display import HTML
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec

# Introduction

An exploration of the mechanics of the AUC - ROC metric in order to develop a deeper understanding of this popular model evaluation metric. 

Will use the [Pima Indians Dataset](https://machinelearningmastery.com/standard-machine-learning-datasets/)

In [ ]:
# No missing values for this dataset
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv")
df.columns =  columns = ['n_pregnancies', 'glucose_conc', 'bp', 'skin_thickness'
                                , 'insulin', 'bmi', 'diabetes', 'age', 'target']



# Manual ROC-AUC computation

1. Synthetic Data Generation:
    - The feature_values generally increase with the probability of label 1, with some noise for non-perfect separation.

2. TPR and FPR Calculation:
    - For each unique feature value as a threshold, we compute TPR and FPR manually by simulating classification decisions based on thresholding.
    
3. AUC Computation using Trapezoidal Rule:
    - For each pair of consecutive ROC points, we calculate the trapezoidal area and add it to the total AUC.
    - For visualization, each trapezoid is shaded, and the boundaries are marked with dotted vertical lines.

In [ ]:
# Step 1: Generate synthetic data
np.random.seed(0)
n = 15 # How many datapoints
# Generate feature values that loosely correlate with the label
feature_values = np.linspace(0, 1, n) + np.random.normal(0, 0.1, n)
# Set labels with a general trend: higher values are more likely to be labeled 1
labels = (feature_values + np.random.normal(0, 0.2, n) > 0.5).astype(int)

# Sort data based on the feature values to simulate model score sorting
sorted_indices = np.argsort(feature_values)
feature_values = feature_values[sorted_indices]
labels = labels[sorted_indices]

# Step 2: Calculate TPR and FPR for ROC without sklearn's built-in functions
# Calculate the true positive rate (TPR) and false positive rate (FPR)
def compute_roc_points(feature_values, labels):
    thresholds = np.unique(feature_values)
    tpr_list = []
    fpr_list = []
    for thresh in thresholds:
        # Classify points based on threshold
        predictions = feature_values >= thresh
        TP = np.sum((predictions == 1) & (labels == 1))
        FP = np.sum((predictions == 1) & (labels == 0))
        FN = np.sum((predictions == 0) & (labels == 1))
        TN = np.sum((predictions == 0) & (labels == 0))
        
        # Calculate TPR and FPR
        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
        
        tpr_list.append(TPR)
        fpr_list.append(FPR)
    
    return np.array(fpr_list), np.array(tpr_list)

# Get FPR and TPR points
fpr, tpr = compute_roc_points(feature_values, labels)

# Step 3: Calculate AUC via the trapezoidal rule and plot each trapezoid
def plot_roc_with_trapezoids(fpr, tpr):
    auc = 0.0
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', marker='o', label='ROC curve')
    
    # Calculate area under the curve using trapezoids and plot them
    for i in range(1, len(fpr)):
        x1, x2 = fpr[i-1], fpr[i]
        y1, y2 = tpr[i-1], tpr[i]
        
        # Compute area of the trapezoid
        trap_area = (x2 - x1) * (y1 + y2) / 2
        auc += trap_area
        
        # Plot trapezoid for visualization
        plt.fill_between([x1, x2], [y1, y2], color='orange', alpha=0.3)
        plt.vlines(x=[x1, x2], ymin=0, ymax=[y1, y2], color='gray', linestyles='dotted')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')
    plt.xlabel("False Positive Rate (FPR)")
    plt.ylabel("True Positive Rate (TPR)")
    plt.title(f"ROC Curve with Trapezoids (AUC ≈ {auc:.2f})")
    plt.legend(loc="lower right")
    plt.show()
    
    return auc

# Plot and compute AUC
auc_score = plot_roc_with_trapezoids(fpr, tpr)
print(f"Computed AUC: {auc_score:.4f}")


In [ ]:
print(roc_auc_score(labels, feature_values))

Manual computation aligns with the result obtained via formula

# Animation

In [15]:
def generate_synthetic_data(n=15, seed=0):
    """Generate synthetic data with some correlation between feature and label"""
    np.random.seed(seed)
    feature_values = np.linspace(0, 1, n) + np.random.normal(0, 0.1, n)
    labels = (feature_values + np.random.normal(0, 0.2, n) > 0.5).astype(int)
    
    # Sort data based on the feature values in DESCENDING order
    sorted_indices = np.argsort(feature_values)[::-1]
    return feature_values[sorted_indices], labels[sorted_indices]

def compute_tpr_fpr(feature_values, labels, threshold_idx):
    """Compute TPR and FPR for a given threshold index"""
    threshold = feature_values[threshold_idx]
    predictions = feature_values >= threshold
    
    TP = np.sum((predictions == 1) & (labels == 1))
    FP = np.sum((predictions == 1) & (labels == 0))
    FN = np.sum((predictions == 0) & (labels == 1))
    TN = np.sum((predictions == 0) & (labels == 0))
    
    TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
    
    return FPR, TPR, TP, FP, FN, TN

def create_roc_animation():
    """Create and return the ROC curve animation"""
    # Generate data
    feature_values, labels = generate_synthetic_data()
    
    # Create figure with custom layout
    fig = plt.figure(figsize=(15, 12))
    gs = GridSpec(2, 2, height_ratios=[1, 1.5], width_ratios=[1, 1], figure=fig)
    
    # Create axes
    ax_scatter = fig.add_subplot(gs[0, 0])  # Top left - scatter plot
    ax_equations = fig.add_subplot(gs[0, 1])  # Top right - equations
    ax_roc = fig.add_subplot(gs[1, :])  # Bottom - ROC curve
    
    plt.subplots_adjust(hspace=0.3, wspace=0.3)
    
    # Storage for ROC curve points and AUC
    fpr_values = []
    tpr_values = []
    auc_score = [0]
    
    # Create threshold line that will be updated
    threshold_line = ax_scatter.axvline(x=0, color='black', linestyle=':', visible=False)
    
    def init():
        """Initialize animation"""
        # Clear axes
        ax_scatter.clear()
        ax_roc.clear()
        ax_equations.clear()
        
        # Setup scatter plot
        ax_scatter.set_xlabel('Feature Values')
        ax_scatter.set_ylabel('Labels')
        ax_scatter.set_title('Feature Values vs Labels\n(Examining thresholds from highest to lowest)')
        ax_scatter.set_ylim(-0.1, 1.1)
        
        # Create scatter plot with colors based on labels
        colors = ['blue' if label == 0 else 'red' for label in labels]
        scatter = ax_scatter.scatter(feature_values, labels, c=colors)
        
        # Add legend to scatter plot
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', markerfacecolor='red', 
                  label='Label 1', markersize=10),
            Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', 
                  label='Label 0', markersize=10)
        ]
        ax_scatter.legend(handles=legend_elements)
        
        # Setup equations plot
        ax_equations.set_title('Current Metrics')
        ax_equations.axis('off')
        
        # Setup ROC plot
        ax_roc.plot([0, 1], [0, 1], 'k--', label='Random classifier')
        ax_roc.set_xlabel('False Positive Rate (FPR)')
        ax_roc.set_ylabel('True Positive Rate (TPR)')
        ax_roc.set_title('ROC Curve Construction')
        ax_roc.grid(True)
        ax_roc.legend(loc='lower right')
        ax_roc.set_xlim(-0.05, 1.05)
        ax_roc.set_ylim(-0.05, 1.05)
        
        # Reset storage
        fpr_values.clear()
        tpr_values.clear()
        auc_score[0] = 0
        
        # Create fresh threshold line
        nonlocal threshold_line
        threshold_line.remove()
        threshold_line = ax_scatter.axvline(x=0, color='black', linestyle=':', visible=False)
        
        return [threshold_line]

    def animate(frame):
        """Update animation for each frame"""
        # Clear previous equations
        ax_equations.clear()
        ax_equations.set_title('Current Metrics')
        ax_equations.axis('off')
        
        if frame > 0:  # Skip first frame (empty plot)
            current_threshold = feature_values[frame-1]
            
            # Update threshold line position and make it visible
            threshold_line.set_xdata([current_threshold, current_threshold])
            threshold_line.set_visible(True)
            
            # Add threshold value annotation to scatter plot
            if hasattr(animate, 'threshold_text'):
                animate.threshold_text.remove()
            animate.threshold_text = ax_scatter.text(
                current_threshold, -0.05, 
                f'Threshold value\n{current_threshold:.2f}',
                ha='center', va='top'
            )
            
            # Compute metrics
            fpr, tpr, TP, FP, FN, TN = compute_tpr_fpr(feature_values, labels, frame-1)
            fpr_values.append(fpr)
            tpr_values.append(tpr)
            
            # Update equations
            ax_equations.text(0.5, 0.9,
                            f'Current Threshold: {current_threshold:.2f}',
                            ha='center', va='center', fontsize=12)
            ax_equations.text(0.5, 0.7, 
                            f'TPR = {TP} / ({TP} + {FN}) = {tpr:.2f}',
                            ha='center', va='center', fontsize=12)
            ax_equations.text(0.5, 0.3,
                            f'FPR = {FP} / ({FP} + {TN}) = {fpr:.2f}',
                            ha='center', va='center', fontsize=12)
            
            # Plot ROC curve points
            if len(fpr_values) > 1:
                # Calculate and plot trapezoid
                x1, x2 = fpr_values[-2], fpr_values[-1]
                y1, y2 = tpr_values[-2], tpr_values[-1]
                trap_area = (x2 - x1) * (y1 + y2) / 2
                auc_score[0] += trap_area
                
                # Fill trapezoid
                ax_roc.fill_between([x1, x2], [y1, y2], color='orange', alpha=0.3)
                # Add vertical lines
                ax_roc.vlines(x=[x1, x2], ymin=0, ymax=[y1, y2], 
                          color='gray', linestyles='dotted')
            
            # Plot ROC curve (maroon color)
            ax_roc.plot(fpr_values[:-1], tpr_values[:-1], color='maroon', marker='o')
            # Plot current point in green
            ax_roc.plot(fpr_values[-1], tpr_values[-1], color='green', marker='o', markersize=10)
            
            # Update AUC text
            if hasattr(animate, 'auc_text'):
                animate.auc_text.remove()
            animate.auc_text = ax_roc.text(0.02, 0.98, f'AUC: {auc_score[0]:.3f}', 
                                      transform=ax_roc.transAxes, 
                                      bbox=dict(facecolor='white', alpha=0.8))
        
        return [threshold_line]

    # Create animation
    anim = FuncAnimation(fig, animate, init_func=init,
                        frames=len(feature_values)+1,
                        interval=1000,
                        blit=False,
                        repeat=False)
    
    plt.close(fig)
    return anim

# Create and display animation
anim = create_roc_animation()
HTML(anim.to_jshtml())

# Ideas

Animation / MP4 like visual showcasing how the:
- Data points are sorted and examined in decreasing order
- TPR + FPR computed at each point (with each point functioning as one "threshold")
- **TPR / FPR** points computed on the ROC curve
- Trapezoid computed

A video of two plots: one above and one below. Plot above is the data points on a feature-label axes. Plot below is the ROC curve. 
- At every tick one data point is examined
- Move a vertical line to indicate the datapoints examined up to this tick
- Update the ROC curve incrementally at each tick, including the new TPR / FPR point + trapezoid

some connection to Gini coeffictient? 
